In [41]:
import pandas as pd
QA_o_f = pd.read_csv("QA_open_500_mapped.csv")
QA_o_f

,questionType,asin,answerTime,unixTime,question,answerType,answer,__index_level_0__,reviews
0,open-ended,B000GU8B28,"Dec 2, 2013",1.385971e+09,I want a small CD player for my bathroom. What...,NaN,"It is 17"" wide, 11"" deep and 3 "" high. If you ...",20648,['Pleased with sound so much we replace old c...
1,open-ended,B000E65DG6,"Feb 6, 2014",1.391674e+09,I have a D battery that's dead and I don't hav...,NaN,Smart chargers need to sense voltage to start ...,17488,"[""Had this for over 10 years. Over the last se..."
2,open-ended,B0000BZOGY,"Oct 11, 2014",1.413011e+09,What are the dimensions of the backpack- what ...,NaN,Hi Laura - Product Dimensions: 17 x 13 x 9 inc...,7102,['I did not care for it. for one my laptop did...
3,open-ended,B00000K2YR,"Dec 6, 2014",1.417853e+09,It is Bluetooth compatible?,NaN,No it isn't.,513,"[""It was expensive, but I liked the small hand..."
4,open-ended,B000AA2RCY,2 days ago,NaN,what is expected shipping time,NaN,It would arrive within 3 -5 business days.,14915,"[""What can I say...it's your standard HDMI cab..."
...,...,...,...,...,...,...,...,...,...
1334,open-ended,B000H866BM,"Aug 6, 2007",1.186384e+09,tom tom battery: does anyone know how to charg...,NaN,Just plug it into your USB port and leave the ...,20962,"[""I've used a Mio C310x for awhile before my s..."
1335,open-ended,B0001XO674,"Mar 2, 2015",1.425283e+09,Are these speakers 200 watts each or 180? The ...,NaN,The speakers are 180 watts,8575,"[""Replaced a set of Aquatronics with these. Mu..."
1336,open-ended,B000BO3CW8,"Apr 10, 2006",1.144652e+09,Does anyone know if this battery will work wit...,NaN,It says in the item description that it will.,15892,"[""Drop-in replacement for the OEM Battery at a..."
1337,open-ended,B000058BCQ,"Apr 20, 2008",1.208675e+09,Will this GPS instrument work in India ? I wan...,NaN,It will work anywhere in the world where you c...,1976,['I was given one of these units as a gift and...


In [43]:
import pickle

with open('generated_answers_open_500.pkl', 'rb') as file:
    generated_answers = pickle.load(file)
with open('retrieved_reviews_open_500.pkl', 'rb') as file:
    retrieved_reviews = pickle.load(file)

In [49]:
import re

match = re.search(r'\[Final Answer\](.*)', generated_answers[0], re.DOTALL)
resposta_final = match.group(1).replace('\n', '').strip()
print(resposta_final)

The CD player is a compact, relatively small unit, but specific dimensions are not provided.


In [51]:
QA_o_f['question']

0       I want a small CD player for my bathroom. What...
1       I have a D battery that's dead and I don't hav...
2       What are the dimensions of the backpack- what ...
3                             It is Bluetooth compatible?
4                          what is expected shipping time
                              ...                        
1334    tom tom battery: does anyone know how to charg...
1335    Are these speakers 200 watts each or 180? The ...
1336    Does anyone know if this battery will work wit...
1337    Will this GPS instrument work in India ? I wan...
1338    Logitech Quickcam for Notebooks Deluxe: For an...
Name: question, Length: 1339, dtype: object

In [53]:
qa_context_dict = {
    'user_input': [],
    'response': [],
    'reference': [],
    'retrieved_contexts': []
}
converted_samples = []
for i in range(len(generated_answers)):
    match = re.search(r'\[Final Answer\](.*)', generated_answers[i], re.DOTALL)
    final_answer = match.group(1).replace('\n', '').strip()
    
    qa_context_dict['user_input'].append(QA_o_f.iloc[i]['question'])
    qa_context_dict['reference'].append(QA_o_f.iloc[i]['answer'])
    qa_context_dict['retrieved_contexts'].append(retrieved_reviews[i])
    qa_context_dict['response'].append(final_answer)

In [57]:
import ast

In [104]:
import requests
import json
from nltk.tokenize import sent_tokenize
context_relevance_prompt = (
    "Please extract relevant sentences from the provided context that can potentially "
    "help answer the following question. The context is product reviews and the question is about the product." 
    "If no relevant sentences are found, or if you "
    "believe the question cannot be answered from the given context, return the phrase "
    "'Insufficient Information'. While extracting candidate sentences you're not allowed "
    "to make any changes to sentences from given context."
    "Use the following format:\n\n"\
    "[\"SENTENCE 1\", \"SENTENCE 2\", \"SENTENCE 3\"]"
)
url = "https://llama.venturus.org.br/v1/chat/completions"

headers = {
    "cf-access-token": token,
    "Content-Type": "application/json"
}
context_relevance_scores = []

for i in range(len(qa_context_dict['user_input'])):
    answer = qa_context_dict['reference'][i]
    original_question = qa_context_dict['user_input'][i]
    context = qa_context_dict['retrieved_contexts'][i]
    context = " ".join(context)
    context = re.sub(r'<[^>]+>', '', context)
    # Remover links (URLs e domínios)
    context = re.sub(r'\b(?:https?://|www\.)\S+\b', '', context)
    context = re.sub(r'\b\w+\.\w+\b', '', context)
    print(i)
    # Contar o número total de sentenças no contexto
    total_sentences = len(sent_tokenize(context))
    # Dados da requisição
    data = {
        "model": "tgi",
        "messages": [
            {"role": "system", "content": context_relevance_prompt},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {original_question}"},
            {"role": "assistant", "content": ""}
        ],
        "temperature": 0.7,  # Configurado para máxima repetibilidade
        "max_tokens": 1000,
        "stream": False
    }
    
    # Faz a requisição POST com streaming
    response = requests.post(url, headers=headers, json=data, stream=False)
    #print(json.loads(response.text)['choices'][0]['message']['content'])
    try:
        outputs = json.loads(response.text)['choices'][0]['message']['content']
        # Extrair as sentenças relevantes retornadas pelo modelo
        extracted_sentences = outputs #[0]["generated_text"][-1]['content']
        # Caso o modelo retorne "Insufficient Information"
        if extracted_sentences.strip() == "Insufficient Information":
            score = 0.0
        else:
            # Caso contrário, contar o número de sentenças extraídas
            extracted_sentences = sent_tokenize(extracted_sentences)
            relevant_sentences_count = len(extracted_sentences)
            score = relevant_sentences_count / total_sentences
        context_relevance_scores.append(score)
        print(f"Question: {original_question}: \n Sentences: {extracted_sentences} \n{score} context relevance score")
    except json.JSONDecodeError:
        print(f"Question: {original_question}")
        print(f"Context: {context}")
        context_relevance_scores.append(0.0)  # Adiciona um valor padrão para esta iteração e continua
    except Exception as e:
        print("Erro inesperado:", e)
        context_relevance_scores.append(0.0)

0
Question: I want a small CD player for my bathroom. What are the dimensions of this player?: 
 Sentences: ['["Small enough to be unobtrusive", "a single-disc player with only basic functionality the TEAC could probably also afford to be a bit smaller in size, with less of a footprint and about half the height, but at a time when low-cost, dedicated, single-disc CD players are a rare commodity, this seems like a minor quibble"]'] 
0.041666666666666664 context relevance score
1
Question: I have a D battery that's dead and I don't have a dumb charger to bring the charge up enough for this to recognize and charge it. How can I charge it?: 
 Sentences: ['["If the battery is extremely discharged the device will not “see it”, and the cell will not charge.', '", "In that case, I remove the battery from the charger, hook the cell up to a DC power supply and connect it to ~5V for a couple of seconds so that it has a small level of power.', '", "Then I return the battery to the charger and it w

In [110]:
import numpy as np
np.mean(context_relevance_scores), np.std(context_relevance_scores), np.median(context_relevance_scores)

(0.09862751291973243, 0.06104728958461962, 0.09090909090909091)

In [114]:
np.max(context_relevance_scores)

0.45

In [85]:
import requests
import json

url = "https://llama.venturus.org.br/v1/chat/completions"

headers = {
    "cf-access-token": token,
    "Content-Type": "application/json"
}
faithfulness_prompt_decompose = (
    "Given a question and answer, create one or more statements from each sentence in the given answer. "
    "The statements must be returned in the following strict format: "
    "a JSON list with each sentence being a string, like this: "
    "[\"SENTENCE 1\", \"SENTENCE 2\", \"SENTENCE 3\"]. "
)
faithfulness_prompt_verify = (
    "Consider the given context and following statements, then determine whether they are supported by the information present "
    "in the context. Provide a brief explanation for each statement before arriving at the verdict (Yes/No). Provide a final "
    "verdict for each statement in order at the end in the given format. Do not deviate from the specified format."
    
)

#faithfulness_scores = []
for i in range(253, len(qa_context_dict['user_input'])+253):
    print(i)
    context = qa_context_dict['retrieved_contexts'][i]
    context = " ".join(context)
    context = re.sub(r'<[^>]+>', '', context)
    question = qa_context_dict['user_input'][i]
    answer = qa_context_dict['response'][i]
    # Passo 1: Gerar declarações a partir da resposta
    decompose_messages = {
        "model": "tgi",
        "messages": [
            {"role": "system", "content": faithfulness_prompt_decompose},
            {"role": "user", "content": f"question: {question}\nanswer: {answer}"},
            {"role": "assistant", "content": ""}
        ],
        "temperature": 0.7,  # Configurado para máxima repetibilidade
        "max_tokens": 1000,
        "stream": False
    }
    # Faz a requisição POST com streaming
    response = requests.post(url, headers=headers, json=decompose_messages, stream=False)
    # Extrair as declarações geradas
    try:
        statements = json.loads(response.text)['choices'][0]['message']['content']
        #print(statements)
        # Extrair a parte que contém a lista de sentenças
        start_index = statements.find('[')  # Encontrar a posição do início da lista
        end_index = statements.find(']') + 1  # Encontrar a posição do fim da lista
        # Obter apenas o texto da lista
        list_str = statements[start_index:end_index]
        # Tentar converter o texto para uma lista
    except json.JSONDecodeError:
        print('Erro JSONDecodeError')
        faithfulness_scores.append(0.0)
    try:
        statements = ast.literal_eval(list_str)  # Converte o texto para uma lista
    except Exception as e:
        print(f"Erro ao converter a string para lista: {e}")
        statements = []  # Caso haja erro, assume uma lista vazia
    # Dados da requisição
    verify_messages = {
        "model": "tgi",
        "messages": [
            {"role": "system", "content": faithfulness_prompt_verify},
            {"role": "user", "content":  f"Context: {context}\n\nStatements:\n" + "\n".join(statements)},
            {"role": "assistant", "content": ""}
        ],
        "temperature": 0.7,  # Configurado para máxima repetibilidade
        "max_tokens": 1000,
        "stream": False
    }
    # Faz a requisição POST com streaming
    response = requests.post(url, headers=headers, json=verify_messages, stream=False)
    # Extrair as verificações de suporte das declarações
    try:
        verification_results = json.loads(response.text)['choices'][0]['message']['content']
        verified_statements = verification_results.splitlines()
        # Contar o número de afirmações que receberam o veredito "Sim"
        supported_count = sum(1 for result in verified_statements if "yes" in result.lower())
        # Passo 3: Calcular a pontuação de "Faithfulness"
        if len(statements)>0:
            faithfulness_score = supported_count / len(statements)
            faithfulness_scores.append(faithfulness_score)
        else:
            faithfulness_scores.append(0.0)
        print(f"Answer {answer}: \n{faithfulness_score} faithfulness score")
    except json.JSONDecodeError:
        print('Erro JSONDecodeError')
        faithfulness_scores.append(0.0)

253
Erro JSONDecodeError
Answer The coaxial is not marked as "in" or "out," but you can determine the correct orientation by mounting the product with the logo upside down and the coaxial cable and phone lines at the top. Alternatively, you can slide off the plastic collar attachment to reveal the input/output labels for the phone and cable jacks.: 
0.0 faithfulness score
254
Answer You will be able to see the planets in our solar system, but the level of detail and color may vary. The product may show the rings of Saturn, the red belts around Jupiter, and the Galilean moons, but it may not provide close-up detailed views of the planets. Using a lens kit or filters may enhance the viewing experience, but the product's performance may be affected by the quality of the lenses.: 
1.0 faithfulness score
255
Answer The Canon S400 can handle a maximum compact flash (CF) capacity of at least 1GB, as evidenced by the successful use of a 1GB Lexar CF card.: 
1.0 faithfulness score
256
Answer Fo

IndexError: list index out of range

In [92]:
import numpy as np
np.mean(faithfulness_scores), np.std(faithfulness_scores), np.median(faithfulness_scores)

(0.8299356041348924, 0.4147516394510828, 1.0)